In [1]:
# Load necessary packages into Python
from getpass import getpass
from netrc import netrc
import time
import os
import requests

# ----------------------------------USER-DEFINED VARIABLES--------------------------------------- #
# Instead of using argparse, we directly set variables
saveDir = r"D:/VUB/_data/modis_ET" # Replace with your desired directory path
files = 'https://e4ftl01.cr.usgs.gov/MOLT/MOD17A2HGF.061/2000.01.01/MOD17A2HGF.A2000001.h10v07.061.2020125212257.hdf'  # Replace with your file or URL

prompts = ['Enter NASA Earthdata Login Username (or create an account at urs.earthdata.nasa.gov): ',
           'Enter NASA Earthdata Login Password: ']

# ---------------------------------SET UP WORKSPACE---------------------------------------------- #
# Create a list of files to download based on input type of files above
if files.endswith('.txt') or files.endswith('.csv'):
    with open(files, 'r') as file:
        fileList = file.readlines()  # If input is a textfile with file URLs
elif isinstance(files, str):
    fileList = [files]  # If input is a single file URL

# Generalize download directory
if not saveDir.endswith(('/','\\')):
    saveDir += os.sep

urs = 'urs.earthdata.nasa.gov'  # Address to call for authentication

# --------------------------------AUTHENTICATION CONFIGURATION----------------------------------- #
# Determine if netrc file exists, and if so, if it includes NASA Earthdata Login Credentials
try:
    netrcDir = os.path.expanduser("~/.netrc")
    credentials = netrc(netrcDir).authenticators(urs)

except FileNotFoundError:
    homeDir = os.path.expanduser("~")
    netrcFilePath = os.path.join(homeDir, ".netrc")
    
    with open(netrcFilePath, 'w') as netrcFile:
        netrcFile.write(f"machine {urs}\n")
        netrcFile.write(f"login {getpass(prompt=prompts[0])}\n")
        netrcFile.write(f"password {getpass(prompt=prompts[1])}\n")
    
    os.chmod(netrcFilePath, 0o600)  # Set file permissions

# Determine OS and edit netrc file if it exists but is not set up for NASA Earthdata Login
except TypeError:
    homeDir = os.path.expanduser("~")
    with open(netrcDir, 'a') as netrcFile:
        netrcFile.write(f"\nmachine {urs}\n")
        netrcFile.write(f"login {getpass(prompt=prompts[0])}\n")
        netrcFile.write(f"password {getpass(prompt=prompts[1])}\n")

# Delay for up to 1 minute to allow user to submit username and password before continuing
tries = 0
while tries < 30:
    try:
        credentials = netrc(netrcDir).authenticators(urs)
        break
    except:
        time.sleep(2.0)
    tries += 1

# -----------------------------------------DOWNLOAD FILE(S)-------------------------------------- #
# Loop through and download all files to the directory specified above, and keeping same filenames
for f in fileList:
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    
    saveName = os.path.join(saveDir, f.split('/')[-1].strip())
    
    # Create and submit request and download file
    with requests.get(f.strip(), verify=False, stream=True, auth=(credentials[0], credentials[2])) as response:
        if response.status_code != 200:
            print(f"{f.split('/')[-1].strip()} not downloaded. Verify that your username and password are correct in {netrcDir}")
        else:
            response.raw.decode_content = True
            content = response.raw
            with open(saveName, 'wb') as d:
                while True:
                    chunk = content.read(16 * 1024)
                    if not chunk:
                        break
                    d.write(chunk)
            print(f'Downloaded file: {saveName}')
    #time.sleep(1.0)


c:\Users\kart\anaconda3\envs\lp_daac_download\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\Users\kart\anaconda3\envs\lp_daac_download\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\Users\kart\anaconda3\envs\lp_daac_download\lib\site-packages\urllib3\connectionpool.py:857: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\Users\kart\anaconda3\envs\lp_daac_download\lib

Downloaded file: D:/VUB/_data/modis_ET\MOD17A2HGF.A2000001.h10v07.061.2020125212257.hdf
